# Read from Excel to CSV

In [3]:
import pandas as pd

# Read Excel file into a pandas dataframe object
f = pd.read_excel(r'data/17766_12_data.xlsx')

# Convert dataframe to CSV format and save to disk
# Use index=None to exclude row numbers from output
f.to_csv(r'data/17766_12_data.csv', index=None)

# Process
### Load, Group, Rename, Save

In [15]:
import pandas as pd

# Read csv file and only keep necessary columns, handle missing values, and fix data types
# RAH: radius of the tree in radius of the tree in micrometer
# RAZ_kum: cumulative radial increment of the tree (in micrometres) since April 1, 2012
df = pd.read_csv(r"data/17766_12_data.csv",
                 usecols=['year', 'DOY', 'dendroNr', 'Art', 'RAH', 'RAZ_kum', 'hour'],
                 na_values=['.'],
                 dtype={'RAH': float, 'RAZ_kum': float}
                 )

# Convert year column from Excel string to integer
df['year'] = df['year'].map(lambda s: s[:4]).astype(int)

# Group the data by unique dendroNr, year, DOY, and hour and rename columns
df = (
    df.groupby(['dendroNr', 'year', 'DOY', 'hour'])
    .agg({'Art': 'first', 'RAZ_kum': 'mean', })
    .rename(columns={'Art': 'species', 'RAZ_kum': 'growth', })
    .reset_index()
)

# Filter the data by removing completely missing values and selecting specific species and timeframes
df = df[
    ~df['growth'].isna()
    & df['species'].isin(['Es', 'Bu', 'BAh'])
    & (
            ((df['year'] == 2012) & (df['DOY'] >= 92) & (df['DOY'] <= 274))
            | ((df['year'] == 2013) & (df['DOY'] >= 91) & (df['DOY'] <= 273))
    )
    ]

# Reset DOY and growth to zero-start in the Series
df = (
    df.groupby(['dendroNr', 'year'], group_keys=True)
    .apply(
        lambda x: x.assign(
            DOY=x['DOY'] - x['DOY'].iloc[0], growth=x['growth'] - x['growth'].iloc[0]
        )
    )
    .reset_index(drop=True)
)

# Change data types for specific columns
df['year'] = df['year'].astype('category')
df['species'] = pd.Categorical(df['species'], categories=['Bu', 'Es', 'BAh'], ordered=True)
df['species'] = df['species'].cat.rename_categories(["Beech", "Ash", "Sycamore"])

# Merge tree and year
df.dendroNr = (
    df[["dendroNr", "year"]].astype(str).agg("_".join, axis=1)
)
df = df.drop("year", axis=1)

# Merge DOY
df.DOY = df.DOY.mul(48) + df.hour.mul(2)
df['DOY'] = df['DOY'].astype('int')

# Save processed data to feather file
df.to_feather(r'data/17766_12_D.feather')
df

,dendroNr,DOY,hour,species,growth
0,HA_001_2012,0,0.0,Beech,0.000
1,HA_001_2012,1,0.5,Beech,0.000
2,HA_001_2012,2,1.0,Beech,0.000
3,HA_001_2012,3,1.5,Beech,0.000
4,HA_001_2012,4,2.0,Beech,0.000
...,...,...,...,...,...
1457969,SH_076_2013,8779,21.5,Beech,3902.797
1457970,SH_076_2013,8780,22.0,Beech,3902.797
1457971,SH_076_2013,8781,22.5,Beech,3903.275
1457972,SH_076_2013,8782,23.0,Beech,3903.275
